In [1]:
#Libraries
import zipfile, os
import shutil
import tarfile
import re
import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import os
import json
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from pdfrw import PdfReader
import numpy as np
from nltk import ngrams
import scispacy
import spacy

## Common Functions to all publication

In [5]:
#NCT Pattern Filter
def NCT_filter(section_dict):
    NCT_dict={}
    for i in range(0,len(list(section_dict.keys()))):
        #print(i)
        if len(re.findall(r'(NCT[0-9]+)'.format(11),str(list(section_dict.values())[i])))>0:
               NCT_dict[list(section_dict.keys())[i]]=list(section_dict.values())[i]
    return NCT_dict

In [6]:
# URL Extraction
from urlextract import URLExtract
def URL_Extract(text_to_extract):
    extractor = URLExtract()
    urls = extractor.find_urls(text_to_extract)
    return urls

In [7]:
# To convert into JSON format
def data_to_json(abstractdataframe,othersectiondataframe,column_name_for_json,XML_file_list,publisher_name):
    json_list=[]
    xml_names_list=[]
    NCT_final_list=[]
    publisher_list=[]
    for xml_name in XML_file_list:
        NCT_total_list=[]
        json_text=""
        xml_df = abstractdataframe.groupby('XML').get_group(xml_name)
        xml_df1 = othersectiondataframe.groupby('XML').get_group(xml_name)        
        #print(dataframename)
        for xml_no in range(0,len(xml_df)):
            if (xml_df['Section Name'].iloc[xml_no]!='NA'):
                json_text = json_text+xml_df[[ 'Section Name', 'NCT Number in Section',
               'Sentence Containing NCT Number', 'Supporting word', 'URL_found']].iloc[xml_no].to_json()+";;;;"
            #To get the NCT List
            if xml_df['NCT Number in Section'].iloc[xml_no] not in NCT_total_list and xml_df['NCT Number in Section'].iloc[xml_no]!='NA':
                NCT_total_list.append(xml_df['NCT Number in Section'].iloc[xml_no])
        
        for xml_no1 in range(0,len(xml_df1)):
            if (xml_df1['Section Name'].iloc[xml_no1]!='NA'):
                json_text = json_text+xml_df1[[ 'Section Name', 'NCT Number in Section',
                'Sentence Containing NCT Number', 'Supporting word', 'URL_found']].iloc[xml_no1].to_json()+";;;;"
            #To get the NCT List
            if xml_df1['NCT Number in Section'].iloc[xml_no1] not in NCT_total_list and xml_df1['NCT Number in Section'].iloc[xml_no1]!='NA':
                NCT_total_list.append(xml_df1['NCT Number in Section'].iloc[xml_no1])
        
        if NCT_total_list==[]:
            NCT_total_list=""
        NCT_final_list.append(NCT_total_list)
        json_list.append(json_text)
        xml_names_list.append(xml_name)
        publisher_list.append(publisher_name)

    json_df = pd.DataFrame(list(zip(xml_names_list,publisher_list,json_list,NCT_final_list)),
                   columns =['XML','Publisher',column_name_for_json,'NCT Full List'])

    return json_df

## Elsevier Functions

In [8]:
# Get Abstrsct data from Article
def get_from_abstract_Elsevier(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.elsevier.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.elsevier.com/xml/common/schema}abstract"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                        abstract_title = elem.text
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}simple-para":
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [9]:
#Get Body from article
def get_sections_from_body_Elsevier(SourceName):
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.elsevier.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.elsevier.com/xml/ja/schema}body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                        section_title = elem.text
                    if elem.tag == "{http://www.elsevier.com/xml/common/schema}para":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
                            
        return section_dict

In [14]:
#To Validate and Get NCT from Abstract
def get_nct_from_abstract_Elsevier(DirectoryPath):
    bodycount = 0 
    count=0
    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    check_list=[]
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.elsevier.com' in shorttext:
                    dd=NCT_filter(get_from_abstract_Elsevier(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #count=count+1

                        #To Verify publication NCT
                        for i in range(0,len(list(dd.keys()))):             
                            NCT_refresh=[] 
                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                            NCT_refresh=' '.join(NCT_refresh).split()
                            #print(NCT_refresh)
                            if len(NCT_refresh)>0:
                                for search_word in NCT_refresh:
                                    if len(search_word)==11:                                    
                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            xml_list.append(xml_name)
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)
                                            val_list.append('Abstract')
                                            check_words=['register','started','ongoing','underway'
                                                             ,'reported','ClinicalTrials.gov'
                                                     ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')

                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        val_list.append('NA')
                        check_list.append('NA')
                df1_elsevier=pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'])
        df1_elsevier['URL_found']=df1_elsevier['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
        return df1_elsevier

In [18]:
#To Get NCT from Body
def get_nct_from_body_Elsevier(DirectoryPath):
    bodycount = 0 
    count=0
    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    section_list=[]
    check_list=[]

    NCT_final_list={}
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(name)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.elsevier.com' in shorttext:
                    dd=NCT_filter(get_sections_from_body_Elsevier(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #To Verify publication NCT
                        NCT_total_list=[]
                        for i in range(0,len(list(dd.keys()))): 
                            #print(len(list(dd.keys())))
                            NCT_refresh=[] 

                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                            NCT_refresh=' '.join(NCT_refresh).split() 
                            #NCT_total_list.append(list(set(NCT_refresh)))
                            #print(NCT_refresh)

                            if len(NCT_refresh)>0:

                                for search_word in NCT_refresh:
                                    if len(search_word)==11:
                                        #print(search_word)

                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            #print(proximity_text)
                                            #print(xml_name)
                                            xml_list.append(xml_name)
                                            section_list.append(list(dd.keys())[i])
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)

                                            check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')
                                            ##val_list.append('Yes')
                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                                        #NCT_final_list.append('NA')
                        NCT_dict={xml_name:NCT_total_list}
                        NCT_final_list.update(NCT_dict)
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        section_list.append('NA')
                        check_list.append('NA')
                        NCT_dict={xml_name:'NA'}
                        NCT_final_list.update(NCT_dict)

                df2_elsevier=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'])
    df2_elsevier['URL_found']=df2_elsevier['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
    return df2_elsevier

In [26]:
#To create JSON using data_to_json function
def get_nct_json_Elsevier(DirectoryPath):
    XML_files_unique = get_nct_from_abstract_Elsevier(DirectoryPath)['XML'].unique()
    final_df_elsevier = data_to_json(get_nct_from_abstract_Elsevier(DirectoryPath)
                                     ,get_nct_from_body_Elsevier(DirectoryPath)
                                     ,'NCT_Json',XML_files_unique,'Elsevier')
    return final_df_elsevier

In [25]:
def get_titles_elsevier(DirectoryPath):
    title_list=[]
    journal_list=[]
    xml_title_list=[]
    actual_title_list=[]
    actual_journal_list=[]

    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.elsevier.com' in shorttext:
                    #print(SourceFolder)
                    tree = etree.parse(SourceFolder)
                    roots = tree.getroot()
                    for tag in roots.iter("{http://purl.org/dc/terms/}title"):
                        title_list.append(ElementTree.tostring(tag))
                        xml_title_list.append(xml_name)

                    for tag in roots.iter("{http://prismstandard.org/namespaces/basic/2.0/}publicationName"):
                        journal_list.append(ElementTree.tostring(tag))

    #actual Title
    for body in title_list:
        root = ElementTree.fromstring(body)
        for elem in root.iter():
            #print(elem.text)
            actual_title_list.append(elem.text)   

    #actual Journal
    for body in journal_list:
        root = ElementTree.fromstring(body)
        for elem in root.iter():
            #print(elem.text)
            actual_journal_list.append(elem.text)  

    title_df_elsevier = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
                   columns =['XML','Title','Journal Title'])
    return title_df_elsevier

In [27]:
def get_final_data_Elsevier(DirectoryPath):
    df_to_merge_elsevier = pd.merge(get_titles_elsevier(DirectoryPath),get_nct_json_Elsevier(DirectoryPath))
    return df_to_merge_elsevier

### Final Elsevier Data (Execute Elsevier functions and Run below cell for Elsevier Output 

In [28]:
# Provide the path of xml files folder
get_final_data_Elsevier('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/')

XML  \
0   014a758a-0be7-4af8-a0f0-1291f198d303.xml   
1   099b9fa2-e041-491c-b486-1a3d534e2eee.xml   
2   0b12daf2-1c3d-41e1-afc1-6e176874c47a.xml   
3   0eb39614-9aab-4135-8acd-7bf80ab0cba7.xml   
4   176db363-c021-4d29-87c7-023169200fdf.xml   
5   1eda660e-600f-4f98-9758-137b9878e7b7.xml   
6   28abfbe3-6332-4df1-ac04-88f33643b81e.xml   
7   30c3d283-6acd-409c-91c1-952fffdc8515.xml   
8   343fc867-b415-4e27-8352-d93a0f3f9c3a.xml   
9   386193d7-e9f2-45d4-80bf-187d1b8c1138.xml   
10  3dd44642-f13a-4cca-83cd-ab262455c5a2.xml   
11  3dee856b-4df5-47a4-bdc5-f42d6586e45a.xml   
12  4069a286-97a8-4d3f-a6a7-5f3a767e5105.xml   
13  427054e3-f9d2-469e-bbd2-5ea132b07a6c.xml   
14  42a2b29f-4eb9-4dd1-a757-b847d348b97f.xml   
15  44eb689c-2e51-400b-ada9-2b6f3fb083e0.xml   
16  4c527b19-40c1-4b09-812c-7817324e4e66.xml   
17  5177a8c0-2ebe-4cef-b5c0-6f2890695064.xml   
18  52feaec4-9bce-4ca1-9fd0-6a2c201020f2.xml   
19  5304e59e-0eaf-480e-a51a-04139abd9913.xml   
20  53734523-9df0-460f-972b-3cd7934e1169.xml   
21  5445f1b8-9ded-478a-a835-076bf5758808.xml   
22  54dadc79-920f-4af3-996e-bc9f2cb0f081.xml   
23  5aeb5591-ac1a-4494-b4c8-c9fc5eff2d29.xml   
24  5f14a655-c955-4d3c-a3e6-352fa8935e32.xml   
25  63d3c516-49c0-4b4c-802f-c9b2977bf6e0.xml   
26  6c63b332-015a-4769-bd9f-c39d0f0d5f1e.xml   
27  772b5a08-d556-4fa2-92d5-8c2cc276c0ba.xml   
28  7820b0bb-2cac-454c-8754-c68531076407.xml   
29  7a2ea7cc-d5a6-4928-9ab5-29d366b541ce.xml   
..                                       ...   
31  90e18dfb-84fb-4cc5-af46-15fb6550066c.xml   
32  99b3a468-44a5-4ee8-9f0d-b5e738e6b055.xml   
33  9de73805-a761-4514-b6d1-15c11f7eb20e.xml   
34  a045cf42-352c-4cea-b8b5-96bb8a91e0c6.xml   
35  a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml   
36  aba5f501-aaad-4f67-b8d8-bd01da5d0ec7.xml   
37  aec0be4c-a4e6-435e-b50f-9ce7edd5a8c2.xml   
38  ba2de9ad-2a4c-4c9b-8978-24b07352d887.xml   
39  ba435e02-670f-4b1e-a41e-22f682473f0c.xml   
40  bbc69df3-927a-4878-b029-a368b8834b35.xml   
41  be0d5eaa-9843-4526-85d9-4e02912a7e03.xml   
42  c121c51b-8191-4c3a-b2f6-942ed4f4c324.xml   
43  c1417907-53ee-4d31-a753-cd008516c324.xml   
44  c20bb69e-4d3c-4dd2-b49c-53c332c921b5.xml   
45  c39ffef4-ef9c-4ed4-951b-6416beb01b25.xml   
46  c512c537-32fb-46b6-adaa-4589799569bc.xml   
47  cb49db09-08c5-4c89-9464-1b2ba3c942e9.xml   
48  ce1115df-1248-419f-a246-e1f6676556bd.xml   
49  d5e39d7f-e7d3-4927-b40b-0aaf9f55cb6f.xml   
50  d76c7bd7-86ce-482b-88c9-22588605c5bc.xml   
51  da25c282-f443-4af2-bd93-4cebdfc99c6a.xml   
52  da32c93e-b332-4e10-b07c-6be7035d575b.xml   
53  dbffee0c-0f49-486a-9770-a603ffbd0bef.xml   
54  e3d459c0-730a-41b4-a5a2-354beb584a8d.xml   
55  e628add2-da6d-4ba9-99e7-fa225a9851ae.xml   
56  ec0e54f8-b9ba-49b1-83f8-7cdb738162d3.xml   
57  f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml   
58  f93cf7ca-9e73-460c-bc04-ca96ea6cb146.xml   
59  fd870d30-e504-4e8a-8c6d-dd895fbfaac8.xml   
60  fea77f7a-eb9c-4093-a235-3732d4ea296c.xml   

                                                Title  \
0   Cancer immunotherapy from biology to nanomedicine   
1   Safety and efficacy of tolebrutinib, an oral b...   
2   Approvals of drugs with uncertain benefit–risk...   
3   Reducing clinical trial risk in multiple scler...   
4   Monoclonal antibodies as immunomodulatory ther...   
5   Mechanisms of action of therapeutic antibodies...   
6   Update in immunosuppressive therapy of myasthe...   
7   Anti-CD20–mediated B-cell depletion in autoimm...   
8   Therapeutic strategies targeting B-cells in mu...   
9   Trial of intrathecal rituximab in progressive ...   
10    B cell targeted therapies in autoimmune disease   
11  Novel Immunotherapeutic Avenues for Rheumatoid...   
12  Second-generation immunotherapeutics in multip...   
13  Clinical translation of immunomodulatory thera...   
14  Ocrelizumab infusion experience in patients wi...   
15  Matching-adjusted indirect treatment compariso...   
16  The COVID-19 pandemic and the use of MS diseas...   
17  Monoclonal Antibodies for Waldenst

## NLM Functions

In [29]:
#Get Abstract data
def get_from_abstract_NLM(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'NLM' in shorttext:
            #print(SourceName)
            
            tree = etree.parse(SourceName)
            roots = tree.getroot()

                
            for tag in roots.iter("abstract"):
                #print(tag)
                tagtext = ElementTree.tostring(tag)
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/abstract|abstract|p|/p|title|/title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                
                #print((tagtext))
                soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext)
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "title":
                        abstract_title = elem.text
                    if elem.tag == "p":
                        #print(elem.text)
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [30]:
#Get Body Data
def get_sections_from_body_NLM(SourceName):
    
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        list_to=[]
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'NLM' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            '''
            for elem in tree.xpath('//italic'):
                parent = elem.getparent()
                parent.remove(elem)
            for elem in tree.xpath('//sub'):
                parent = elem.getparent()
                parent.remove(elem)
            for elem in tree.xpath('//xref'):
                parent = elem.getparent()
                parent.remove(elem)
                #print(etree.tostring(doc))
            '''
            for tag in roots.iter("body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/body|body|p|/p|title|/title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "title":
                        section_title = elem.text
                        
                    if elem.tag == "p":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
        section_dict={k: v for k, v in section_dict.items() if k}                 
        return section_dict

In [31]:
#To Get NCT from Abstract
def get_nct_from_abstract_NLM(DirectoryPath):
    bodycount = 0 
    count=0
    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    check_list=[]
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    dd=NCT_filter(get_from_abstract_NLM(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #count=count+1

                        #To Verify publication NCT
                        for i in range(0,len(list(dd.keys()))):             
                            NCT_refresh=[] 
                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                            NCT_refresh=' '.join(NCT_refresh).split()
                            #print(NCT_refresh)
                            if len(NCT_refresh)>0:
                                for search_word in NCT_refresh:
                                    if len(search_word)==11:                                    
                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            xml_list.append(xml_name)
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)
                                            val_list.append('Abstract')
                                            check_words=['register','started','ongoing','underway'
                                                             ,'reported','ClinicalTrials.gov'
                                                     ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')

                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        val_list.append('NA')
                        check_list.append('NA')
            df1_NLM =pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
    df1_NLM['URL_found']=df1_NLM['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
    return df1_NLM

In [32]:
def get_nct_from_body_NLM(DirectoryPath):

    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    section_list=[]
    check_list=[]

    NCT_final_list={}
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(name)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    dd=NCT_filter(get_sections_from_body_NLM(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #To Verify publication NCT
                        NCT_total_list=[]
                        for i in range(0,len(list(dd.keys()))): 
                            #print(len(list(dd.keys())))
                            NCT_refresh=[] 

                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                            NCT_refresh=' '.join(NCT_refresh).split() 
                            #NCT_total_list.append(list(set(NCT_refresh)))
                            #print(NCT_refresh)

                            if len(NCT_refresh)>0:

                                for search_word in NCT_refresh:
                                    if len(search_word)==11:
                                        #print(search_word)

                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            #print(proximity_text)
                                            #print(xml_name)
                                            xml_list.append(xml_name)
                                            section_list.append(list(dd.keys())[i])
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)

                                            check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')
                                            ##val_list.append('Yes')
                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                                        #NCT_final_list.append('NA')
                        NCT_dict={xml_name:NCT_total_list}
                        NCT_final_list.update(NCT_dict)
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        section_list.append('NA')
                        check_list.append('NA')
                        NCT_dict={xml_name:'NA'}
                        NCT_final_list.update(NCT_dict)

                df2_NLM=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
    df2_NLM['URL_found']=df2_NLM['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
    return df2_NLM

In [33]:
#To create JSON using data_to_json function
def get_nct_json_NLM(DirectoryPath):
    XML_files_unique = get_nct_from_abstract_NLM(DirectoryPath)['XML'].unique()
    final_df_NLM = data_to_json(get_nct_from_abstract_NLM(DirectoryPath)
                                     ,get_nct_from_body_NLM(DirectoryPath)
                                     ,'NCT_Json',XML_files_unique,'NLM')
    return final_df_NLM

In [36]:
def NLM_Titles_XML(RootDir1):
    xml_title_list=[]
    actual_title_list=[]
    actual_journal_list=[]

    for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    #print(SourceFolder)
                    tree = etree.parse(SourceFolder)
                    roots = tree.getroot()


                    for tag in roots.iter('front'):
                        tagtext = ElementTree.tostring(tag) 
                        #print(tagtext)
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/article-title|article-title|front|/front|journal-title|/journal-title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        #print(tagtext)
                        root_1 = ElementTree.fromstring(tagtext)               
                        title_sen=""
                        journal_sen=""
                        for elem in root_1.iter():
                            #print(elem)
                            if elem.tag == "article-title":                    
                                title_sen = title_sen+elem.text
                            if elem.tag =="journal-title":
                                journal_sen= journal_sen+elem.text

                        actual_journal_list.append(journal_sen)
                        actual_title_list.append(title_sen)
                        xml_title_list.append(xml_name)

    title_df_NLM = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
                   columns =['XML','Title','Journal Title'])
    return title_df_NLM

In [34]:
def get_final_data_NLM(DirectoryPath):
    df_to_merge_NLM = pd.merge(NLM_Titles_XML(DirectoryPath),get_nct_json_NLM(DirectoryPath))
    return df_to_merge_NLM

### Final NLM Data (Execute NLM functions and Run below cell for NLM Output)

In [37]:
# Provide the path of xml files folder
get_final_data_NLM('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/')

XML  \
0    01bcf6be-d833-41a9-a7fc-be3bdcf3ac32.xml   
1    01e1d1b1-a8cb-4d75-8792-7b1fe815646e.xml   
2    02be68ce-7530-4882-9a02-d793e6306b43.xml   
3    0306fb29-0c81-49d8-8a0b-33b92ddadd26.xml   
4    05294995-39f2-48b3-bbae-0a23ffad6c7f.xml   
5    068bfff5-8882-4c9b-85ba-6d5dce67aa53.xml   
6    090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml   
7    0aadf051-3352-4a49-8744-631de0697f0a.xml   
8    0bc312b9-a280-4d2c-8ebe-92f48585707f.xml   
9    0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml   
10   0f2f6b6c-7f6d-4b64-92ca-a16999a09c71.xml   
11   15144c1e-5c1d-477e-b0db-437eebe23d97.xml   
12   15914e48-3869-47ec-8f82-f3b13638d8e5.xml   
13   15a5c64c-891b-415c-b706-76d1a45cbdf6.xml   
14   19636024-41d2-4607-86ca-c56892608622.xml   
15   1ba62e19-e0c5-4b43-8f31-01ec63e66ffb.xml   
16   1bd10a42-8aa8-49f4-bd4b-5e59ac175c0c.xml   
17   1c2d5b27-8bf9-476a-83b2-12f7e539bada.xml   
18   1f5ef9dd-d064-4888-babc-f09c721620ea.xml   
19   22adc16b-8685-42ff-aa91-d46f10eb55be.xml   
20   2468cc7b-4c08-46a3-97b4-104a9810cc36.xml   
21   251647c1-05e3-443b-8fbd-e3a01dac1d3c.xml   
22   28ee2704-fb96-4e01-8b1d-3b262b082623.xml   
23   311b274e-c6a7-4c21-bfdb-52664722abff.xml   
24   31bef979-d07b-4332-9438-53dc9a7e7df9.xml   
25   3280ec95-ba06-4699-852e-5eea9a09f9f8.xml   
26   3392759a-b97d-41ba-b077-69207e85bca1.xml   
27   3bca6af4-e13e-4df9-90ad-a5878eadd58e.xml   
28   3bcd9e40-ab01-498a-a86c-17b9971db667.xml   
29   3bcef5b4-c97d-4fb2-921b-44955d3c8156.xml   
..                                        ...   
99   bdae611a-da74-41e0-8e0b-8de922ef0e6c.xml   
100  bdc23e24-543f-4bdd-83be-ed7aa5928745.xml   
101  c3ada49d-0b6c-4664-9b67-25df5c41e290.xml   
102  c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml   
103  c723c457-acd1-4a43-be22-4a796692b495.xml   
104  c798564e-9105-45e4-b29a-73eb09b2b2fd.xml   
105  c7d988be-6699-4a96-8338-497b9984e784.xml   
106  ca148833-6462-410e-9898-bc995b01ad4a.xml   
107  cadcf7c3-3c90-4512-8ecc-e3d7f611cc55.xml   
108  cb8fc27c-c46e-4977-869e-6f344b26d223.xml   
109  cd23a882-bccd-4faa-aadf-73739e1bc736.xml   
110  d0fc120b-34c5-4c77-80df-1899801e59dd.xml   
111  d11309dc-a2fa-4ab6-818a-1e0590ecbc27.xml   
112  dc843218-8a0e-4946-b8b8-0ca4349db24f.xml   
113  ded024a8-3610-4f73-a29b-adec96685b33.xml   
114  dee575b2-2841-4a8f-8055-dfcb5418af48.xml   
115  e0195958-a9dd-47ac-ade8-46e2c93f48e8.xml   
116  e0bc7241-014a-40e3-a7fb-f43ab81b4502.xml   
117  e509b3bf-4e6b-48ee-8cfe-a92deb9771b9.xml   
118  e5b5b6a1-49e3-400f-bf93-4779aa4ad492.xml   
119  e60923f7-cbcc-4412-b7a0-9660e5f1b74b.xml   
120  ec819b82-c4fa-4eb4-a73c-d38376aa456b.xml   
121  ecdc1a17-0574-49be-a092-d7abca902752.xml   
122  f18576d9-ac93-45cb-adff-98efb35dc8b0.xml   
123  f19438e6-8ec2-48eb-a3cd-66d3b5b3dcdc.xml   
124  f2037c20-a62f-420c-a8b7-19837ddfac7c.xml   
125  f8e72c62-988e-4efc-8d07-c8a214c323a4.xml   
126  fb120bc9-b590-4877-aec0-daf8cebf04f3.xml   
127  ff4db333-e929-492d-b335-4e2092417000.xml   
128  ff825821-c3f4-45c4-87ed-82fef03c0ffb.xml   

                                                 Title  \
0    Early and unrestricted access to high-efficacy...   
1    Current and emerging treatment options for aut...   
2    Obinutuzumab induces superior B-cell cytotoxic...   
3    Recent Advances in Monoclonal Antibody Therapi...   
4    Rituximab in Multiple Sclerosis: Are We Ready ...   
5    Invasive Fungal Diseases in Children with Hema...   
6    Monoclonal Antibody Therapies for Hematologica...   
7    Development of humoral and cellular immunologi...   
8    Investigational drugs for treating psoriatic a...   
9    Alemtuzumab for the treatment of relapsing-rem...   
10   Current and emerging monoclonal antibody treat...   
11   Emerging biological therapies to treat acute l...   
12   PCI-32765: a novel Bruton's tyrosine kinase in...   
13   Differential Effects of MS Therapeutics on B C...   
14   Skin-Associated B Cells in the Pathogenesis of...   
15   What's New in the Treatment of Systemic Lupus ...   
16   Novel Molecu

## Wiley Functions

In [38]:
def get_from_abstract_Wiley(SourceName):
    abs_dict = {}  
    abstract_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.wiley.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.wiley.com/namespaces/wiley}abstract"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/ns0:abstract|ns0:abstract|ns0:p|/ns0:p|ns0:title|/ns0:title).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                #print(tagtext)
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    #if elem.tag == "{http://www.wiley.com/namespaces/wiley}section-title":
                    abstract_title = 'Abstract'
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}p":
                        if (elem.text)==None:
                            pass
                        elif abstract_title in abs_dict:
                            abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                        else:
                            abs_dict[abstract_title] = elem.text
                            
        return abs_dict

In [39]:
def get_sections_from_body_Wiley(SourceName):
    section_dict = {}  
    section_title = ""
    with open(SourceName,'rb') as name:
        data = name.read()
        text = str(data)
        shorttext = str(data)[:100]
        if 'www.wiley.com' in shorttext:
            #print(SourceName)
            tree = etree.parse(SourceName)
            roots = tree.getroot()
            for tag in roots.iter("{http://www.wiley.com/namespaces/wiley}body"):
                tagtext = ElementTree.tostring(tag)  
                #print(tagtext)
                tagtext= (str(tagtext, 'UTF-8'))
                cleanr = re.compile(r'(<(?!/ns0:body|ns0:body|ns0:p|/ns0:p|ns0:title|/ns0:title|/table|table).*?>)|(\n)')
                tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                #print(tagtext)
                #soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                #pageText = soup.findAll(text=True)
                #print(pageText)
                root_1 = ElementTree.fromstring(tagtext) 
                
                for elem in root_1.iter():
                    #print(elem)
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":
                        section_title = elem.text
                    if elem.tag == "{http://www.wiley.com/namespaces/wiley}p":
                        if (elem.text)==None:
                            pass
                        elif section_title in section_dict:
                            section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                        else:
                            section_dict[section_title] = elem.text
                            
        return section_dict

In [41]:
def get_nct_from_abstract_Wiley(DirectoryPath):
    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    check_list=[]
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.wiley.com' in shorttext:
                    dd=NCT_filter(get_from_abstract_Wiley(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #count=count+1

                        #To Verify publication NCT
                        for i in range(0,len(list(dd.keys()))):             
                            NCT_refresh=[] 
                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))))
                            NCT_refresh=' '.join(NCT_refresh).split()
                            #print(NCT_refresh)
                            if len(NCT_refresh)>0:
                                for search_word in NCT_refresh:
                                    if len(search_word)==11:                                    
                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):
                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            xml_list.append(xml_name)
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)
                                            val_list.append('Abstract')
                                            check_words=['register','started','ongoing','underway'
                                                             ,'reported','ClinicalTrials.gov'
                                                     ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')

                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        val_list.append('NA')
                        check_list.append('NA')
                df1_wiley=pd.DataFrame(list(zip(xml_list,val_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object).drop_duplicates()
    df1_wiley['URL_found']=df1_wiley['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
    return df1_wiley

In [42]:
def get_nct_from_body_Wiley(DirectoryPath):
    xml_list=[]
    proximity_list=[]
    search_list=[]
    val_list=[]
    section_list=[]
    check_list=[]

    NCT_final_list={}
    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(name)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.wiley.com' in shorttext:
                    dd=NCT_filter(get_sections_from_body_Wiley(SourceFolder)) #Functions  NCT_filter,get_from_abstract
                    #print(dd)
                    if len(dd)>0:
                        #To Verify publication NCT
                        NCT_total_list=[]
                        for i in range(0,len(list(dd.keys()))): 
                            #print(len(list(dd.keys())))
                            NCT_refresh=[] 

                            NCT_refresh.append(' '.join(re.findall(r'(NCT[0-9]+)'.format(11),str(list(dd.values())[i]))) )
                            NCT_refresh=' '.join(NCT_refresh).split() 
                            #NCT_total_list.append(list(set(NCT_refresh)))
                            #print(NCT_refresh)

                            if len(NCT_refresh)>0:

                                for search_word in NCT_refresh:
                                    if len(search_word)==11:
                                        #print(search_word)

                                        for j in range(0,len(re.split(search_word,str(list(dd.values())[i])))-1):

                                            proximity_text =(' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j])).split()[-30:])
                                                             +" "+search_word+" "+
                                                             ' '.join(re.sub("", "",  (re.split(search_word,str(list(dd.values())[i]))[j+1])).split()[:30]))
                                            #print(proximity_text)
                                            #print(xml_name)
                                            xml_list.append(xml_name)
                                            section_list.append(list(dd.keys())[i])
                                            proximity_list.append(proximity_text)
                                            search_list.append(search_word)

                                            check_words=['register','started','ongoing','underway'
                                                         ,'reported','ClinicalTrials.gov'
                                                 ,'identifier','under evaluation']
                                            NCT_verified=[]
                                            for check_word in check_words:
                                                if (len(re.split((check_word).lower(),str(proximity_text).lower())))>1:
                                                    NCT_verified.append(check_word)
                                            if len(list(set(NCT_verified)))>0:
                                                check_list.append(list(set(NCT_verified)))
                                            else:
                                                check_list.append('NA')
                                            ##val_list.append('Yes')
                                    else:
                                        xml_list.append(xml_name)
                                        section_list.append('NA')
                                        proximity_list.append('NA')
                                        search_list.append('NA')
                                        check_list.append('NA')
                                        #NCT_final_list.append('NA')
                        NCT_dict={xml_name:NCT_total_list}
                        NCT_final_list.update(NCT_dict)
                    else:
                        xml_list.append(xml_name)
                        proximity_list.append('NA')
                        search_list.append('NA')
                        section_list.append('NA')
                        check_list.append('NA')
                        NCT_dict={xml_name:'NA'}
                        NCT_final_list.update(NCT_dict)

                df2_wiley=pd.DataFrame(list(zip(xml_list,section_list, search_list, proximity_list,check_list)),
                   columns =['XML','Section Name', 'NCT Number in Section', 'Sentence Containing NCT Number','Supporting word'],dtype=object)
    df2_wiley['URL_found']=df2_wiley['Sentence Containing NCT Number'].apply(lambda x:URL_Extract(str(x)))
    return df2_wiley

In [43]:
#To create JSON using data_to_json function
def get_nct_json_Wiley(DirectoryPath):
    XML_files_unique = get_nct_from_abstract_Wiley(DirectoryPath)['XML'].unique()
    final_df_Wiley = data_to_json(get_nct_from_abstract_Wiley(DirectoryPath)
                                     ,get_nct_from_body_Wiley(DirectoryPath)
                                     ,'NCT_Json',XML_files_unique,'Wiley')
    return final_df_Wiley

In [45]:
def Wiley_Titles_XML(DirectoryPath):
    xml_title_list=[]
    actual_title_list=[]
    actual_journal_list=[]

    for root, dirs, files in os.walk((os.path.normpath(DirectoryPath)), topdown=False):         
        for name in files:
            SourceFolder = os.path.join(root,name)
            #print(SourceFolder)
            xml_name=name
            with open(SourceFolder,'rb') as name:
                title_list=[]
                journal_list=[]
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'www.wiley.com' in shorttext:
                    #print(SourceFolder)
                    tree = etree.parse(SourceFolder)
                    roots = tree.getroot()

                    for tag in roots.iter('{http://www.wiley.com/namespaces/wiley}contentMeta'):
                        tagtext = ElementTree.tostring(tag) 
                        #print(tagtext)
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/ns0:contentMeta|ns0:contentMeta|ns0:publicationMeta|/ns0:publicationMeta|ns0:header|/ns0:header|ns0:title|/ns0:title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        #print(tagtext)
                        root_1 = ElementTree.fromstring(tagtext)               

                        for elem in root_1.iter():
                            if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":                    
                                title_list.append(elem.text)


                    for tag in roots.iter('{http://www.wiley.com/namespaces/wiley}publicationMeta'):
                        tagtext = ElementTree.tostring(tag) 
                        #print(tagtext)
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/ns0:contentMeta|ns0:contentMeta|ns0:publicationMeta|/ns0:publicationMeta|ns0:header|/ns0:header|ns0:title|/ns0:title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        #print(tagtext)
                        root_1 = ElementTree.fromstring(tagtext)               

                        for elem in root_1.iter():
                            if elem.tag == "{http://www.wiley.com/namespaces/wiley}title":                    
                                journal_list.append(elem.text)


                    actual_journal_list.append(journal_list[0])
                    actual_title_list.append(title_list[0])
                    xml_title_list.append(xml_name)

    title_df_wiley = pd.DataFrame(list(zip(xml_title_list, actual_title_list,actual_journal_list)),
                   columns =['XML','Title','Journal Title'])
    return title_df_wiley

In [46]:
def get_final_data_Wiley(DirectoryPath):
    df_to_merge_Wiley = pd.merge(Wiley_Titles_XML(DirectoryPath),get_nct_json_Wiley(DirectoryPath))
    return df_to_merge_Wiley

### Final Wiley Data (Execute Wiley functions and Run below cell for Wiley Output )

In [47]:
# Provide the path of xml files folder
get_final_data_Wiley('C:/Users/GOWRICH1/Downloads/HAWK AI/Processed/')

XML  \
0   0b6c15ad-8f84-4ecf-81cb-d0bad9e7cd99.xml   
1   0dda512f-64cc-471d-8299-7f3156f05b4c.xml   
2   1db7d591-da69-403a-80e4-8e8d4533c747.xml   
3   327d7412-5e45-43cd-b725-282660bb2fe8.xml   
4   3d888bae-1c8d-42cb-973a-508f6691ce89.xml   
5   3ec7266b-2cf2-4abd-8ae0-b8c7de0c1b97.xml   
6   3f4459c8-75db-4947-8d67-56f43e312ade.xml   
7   40d6cd65-f0f7-4bf5-8e34-d4f0526e6785.xml   
8   43e72596-c148-4ae1-9f50-cb7c891f3fa9.xml   
9   46dce873-dc55-4035-9fbb-91659db416ea.xml   
10  48667507-2430-4a22-a2c8-e9a2b881f86c.xml   
11  515b8e6a-a467-4c57-b68b-e8d86562bb28.xml   
12  52fa1ade-c0f3-476a-a1b2-91242bc736ee.xml   
13  534cbdbf-ce82-472e-a89f-975d010b89ca.xml   
14  540651af-e108-4366-b987-1ac669524edc.xml   
15  54e191e9-f777-4f62-a711-542ee2262cc2.xml   
16  5d36c664-61ad-4386-96ec-ebfc167db2a3.xml   
17  634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18  6571c94f-5207-46ab-aa3a-81e06389ee06.xml   
19  6ed15061-93e2-48bd-9bfa-ded9b54d1691.xml   
20  7297f07c-497b-4425-b1dd-177eb3dda1c3.xml   
21  7c046239-703e-47de-90e4-98e4d3cb91c9.xml   
22  8324e21b-ca0c-4ad3-bd6f-7a09becb047e.xml   
23  89d80693-2b17-43b9-ac53-9562ceb25948.xml   
24  93587271-2c84-494b-9a41-682faabe47b2.xml   
25  a8c2531a-62c8-4888-9976-7da1fab5348f.xml   
26  af27a976-c99b-4fe1-bf5e-bbdeb1797611.xml   
27  b42fc71b-a626-4019-87d0-fa364bf88e7d.xml   
28  b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
29  c3acc47f-7596-43d8-97a6-b2827bb2ebce.xml   
30  c4369451-091d-47d4-90e9-0f3c108b89aa.xml   
31  c6321da9-b781-482b-87d8-6dd832c630a5.xml   
32  c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
33  cb74abed-5ed4-4029-9117-078efbce9780.xml   
34  ce310c5a-de31-495c-8f93-ce59a11f4bbf.xml   
35  cf43627e-0109-4bd1-bdae-9c311b8fc5de.xml   
36  d1f8f853-3a1b-493b-920e-609ca5630ee4.xml   
37  d70a6729-b105-4ca6-89da-e4ea1981ebee.xml   
38  e552c93c-8f64-4ca5-8e15-def481a5c29b.xml   
39  e57e9944-53ef-4be9-8a45-2ec3dfd9d1a7.xml   
40  ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
41  f6331576-9051-45b2-aa23-68b826197125.xml   
42  f87a9d96-e2f0-4413-9cab-5361f9d152ec.xml   
43  fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   

                                                Title  \
0   Beyond Randomized Clinical Trials: Use of Exte...   
1   Chronic lymphocytic leukemia: 2022 update on d...   
2   Ofatumumab, the first human anti‐CD20 monoclon...   
3   Outcomes of coronavirus disease 2019 in patien...   
4   A double‐blind, placebo‐controlled study of ri...   
5   Anti‐CD20 monoclonal antibodies and their use ...   
6   Beyond Randomized Clinical Trials: Use of Exte...   
7   Autoimmune cytopenias in chronic lymphocytic l...   
8   Ofatumumab‐associated acute respiratory manife...   
9       Challenges in the pursuit of immune tolerance   
10  The IUPHAR Guide to Immunopharmacology: connec...   
11  Myasthenia gravis and specific immunotherapy: ...   
12  Chronic lymphocytic leukemia: 2022 update on d...   
13  Treatment of systemic lupus erythematosus: new...   
14  Is There a Place for Chimeric Antigen Receptor...   
15     Novel Multiple Sclerosis Drugs in the Pipeline   
16  Blinded continuous monitoring in clinical tria...   
17  Current and emerging disease‐modulatory therap...   
18  Population pharmacokinetics of ofatumumab in p...   
19  Characteristics and follow‐up of postmarketing...   
20  Developing the next generation of monoclonal a...   
21  From bench to bedside: evolving therapeutic ta...   
22  Is There a Place for Chimeric Antigen Receptor...   
23  A changing treatment landscape for multiple sc...   
24  SARS‐CoV‐2 and Multiple Sclerosis: Not All Imm...   
25  Modulating inflammation and neuroprotection in...   
26  New era of therapy for endocrine autoimmune di...   
27                 DMDs in neuroimmunological disease   
28  Therapeutic Application of Monoclonal Antibodi...   
29  Ocrelizumab in relapsing and primary progressi...   
30                         MS treatment and infection   
31  Assessment of the effect of ofatumumab on card...   